## Original code

In [1]:
import cv2
import requests
import os
from IPython.display import HTML, display
from base64 import b64encode
import cv2
from ultralytics import YOLO
from ultralytics.yolo.utils import render_result

# Function for Image Inference
def yolov8_img_inference(image_path, model_path, image_size=640, conf_threshold=0.25, iou_threshold=0.45):

    # Load the YOLO model from the specified path
    model = YOLO(model_path)

    # Set the model's confidence threshold for predictions
    model.overrides['conf'] = conf_threshold

    # Set the Intersection over Union threshold 
    model.overrides['iou'] = iou_threshold

    # Non-Maximum Suppression will not be class-agnostic
    model.overrides['agnostic_nms'] = False

    # Set the maximum number of detections per image
    model.overrides['max_det'] = 1000

    # Read the input image using OpenCV
    image = cv2.imread(image_path)

    # Perform inference on the image
    results = model.predict(image)

    # Render the results on the image
    render = render_result(model=model, image=image, result=results[0])

    return render

ModuleNotFoundError: No module named 'ultralytics.yolo'

In [2]:
# Example usage for image inference
image_path = 'download.png'  # Replace with your image path
model_path = 'model.pt'
rendered_image = yolov8_img_inference(image_path, model_path)
rendered_image

NameError: name 'yolov8_img_inference' is not defined

## Test Model / Load Model

In [3]:
import torch
model_data = torch.load('model.pt', map_location='cpu')
print(type(model_data))
if isinstance(model_data, dict):
    print(list(model_data.keys()))

<class 'dict'>
['date', 'version', 'license', 'docs', 'epoch', 'best_fitness', 'model', 'ema', 'updates', 'optimizer', 'train_args', 'train_metrics', 'train_results']


## New Code Running

In [4]:
from ultralytics import YOLO
import torch
import cv2
import numpy as np

def yolov8_img_inference(image_path, model_path, image_size=640, conf_threshold=0.25, iou_threshold=0.45):
    """
    Perform object detection inference using YOLOv8 model on an image.
    
    Args:
        image_path (str): Path to the input image.
        model_path (str): Path to the YOLOv8 model file.
        image_size (int): Size of the image for inference.
        conf_threshold (float): Confidence threshold for detections.
        iou_threshold (float): IoU threshold for NMS.
        
    Returns:
        numpy.ndarray: Image with detection results drawn on it.
    """
    # Load the image first
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Could not read image at {image_path}")
    
    # Load the model with PyTorch 2.5.0 compatibility
    # PyTorch 2.5.0 handles the weights_only parameter correctly
    model = YOLO(model_path)
    
    # Run inference using the predict method with proper parameters
    results = model.predict(
        source=image,
        imgsz=image_size,
        conf=conf_threshold,
        iou=iou_threshold,
        verbose=False
    )
    
    # Get the rendered image with detections drawn on it
    rendered_image = results[0].plot()
    
    return rendered_image

# Example usage
if __name__ == "__main__":
    image_path = 'download.png'  # Replace with your image path
    model_path = 'model.pt'
    rendered_image = yolov8_img_inference(image_path, model_path)
    
    # Display the image if needed
    # `cv2.imshow("YOLOv8 Detection", rendered_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()`
    
    # Or save the image
    cv2.imwrite("result.jpg", rendered_image)

## Tuning

In [ ]:
from ultralytics import YOLO
import torch
import cv2
import numpy as np
from typing import Dict, Union, Tuple, Optional, Any
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


def load_image(image_path: str) -> np.ndarray:
    """
    Load an image from path and handle errors.
    
    Args:
        image_path: Path to the input image
        
    Returns:
        Image as numpy array
        
    Raises:
        FileNotFoundError: If image cannot be read
    """
    image = cv2.imread(image_path)
    if image is None:
        logger.error(f"Failed to load image from {image_path}")
        raise FileNotFoundError(f"Could not read image at {image_path}")
    return image


def load_model(model_path: str) -> YOLO:
    """
    Load YOLOv8 model from path and handle errors.
    
    Args:
        model_path: Path to the YOLOv8 model file
        
    Returns:
        Loaded YOLO model
        
    Raises:
        FileNotFoundError: If model cannot be found
        RuntimeError: If model loading fails
    """
    try:
        logger.info(f"Loading model from {model_path}")
        model = YOLO(model_path)
        return model
    except FileNotFoundError:
        logger.error(f"Model not found at {model_path}")
        raise
    except Exception as e:
        logger.error(f"Failed to load model: {str(e)}")
        raise RuntimeError(f"Failed to load model: {str(e)}")


def run_inference(
    model: YOLO, 
    image: np.ndarray,
    config: Dict[str, Any]
) -> Any:
    """
    Run inference using YOLOv8 model.
    
    Args:
        model: Loaded YOLO model
        image: Input image as numpy array
        config: Dictionary containing inference parameters
        
    Returns:
        YOLO results object
    """
    try:
        logger.info("Running inference with configuration: %s", config)
        results = model.predict(
            source=image,
            imgsz=config['image_size'],
            conf=config['conf_threshold'],
            iou=config['iou_threshold'],
            verbose=config.get('verbose', False)
        )
        return results
    except Exception as e:
        logger.error(f"Inference failed: {str(e)}")
        raise


def yolov8_img_inference(
    image_path: str, 
    model_path: str, 
    image_size: int = 640, 
    conf_threshold: float = 0.25, 
    iou_threshold: float = 0.45,
    verbose: bool = False,
    return_results: bool = False
) -> Union[np.ndarray, Tuple[np.ndarray, Any]]:
    """
    Perform object detection inference using YOLOv8 model on an image.
    
    Args:
        image_path: Path to the input image
        model_path: Path to the YOLOv8 model file
        image_size: Size of the image for inference
        conf_threshold: Confidence threshold for detections
        iou_threshold: IoU threshold for NMS
        verbose: Whether to print verbose output
        return_results: Whether to return raw results alongside rendered image
        
    Returns:
        If return_results is False: Image with detection results drawn on it
        If return_results is True: Tuple of (rendered image, raw results)
    """
    # Create configuration dictionary
    config = {
        'image_size': image_size,
        'conf_threshold': conf_threshold,
        'iou_threshold': iou_threshold,
        'verbose': verbose
    }
    
    # Load image and model
    image = load_image(image_path)
    model = load_model(model_path)
    
    # Run inference
    results = run_inference(model, image, config)
    
    # Get the rendered image with detections
    rendered_image = results[0].plot()
    
    if return_results:
        return rendered_image, results
    return rendered_image


def visualize_results(image: np.ndarray, title: str = "YOLOv8 Detection", 
                     save_path: Optional[str] = None, display: bool = True) -> None:
    """
    Visualize or save detection results.
    
    Args:
        image: Image with detections drawn
        title: Window title for display
        save_path: Path to save the image (if None, won't save)
        display: Whether to display the image
    """
    if display:
        cv2.imshow(title, image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    if save_path:
        logger.info(f"Saving result to {save_path}")
        cv2.imwrite(save_path, image)


# Example usage
if __name__ == "__main__":
    try:
        image_path = 'download.png'  # Replace with your image path
        model_path = 'model.pt'      # Replace with your model path
        
        # Simple usage
        rendered_image = yolov8_img_inference(image_path, model_path)
        visualize_results(rendered_image, save_path="result.jpg")
        
        # Advanced usage with raw results
        rendered_image, results = yolov8_img_inference(
            image_path, 
            model_path,
            image_size=1280,
            conf_threshold=0.3,
            iou_threshold=0.5,
            return_results=True
        )
        
        # Access detection data from results
        num_detections = len(results[0].boxes)
        logger.info(f"Detected {num_detections} objects")
        
        # Example of extracting and using detection information
        if num_detections > 0:
            classes = results[0].boxes.cls.cpu().numpy()
            confidences = results[0].boxes.conf.cpu().numpy()
            names = results[0].names  # Class name mapping
            
            for i in range(num_detections):
                class_id = int(classes[i])
                class_name = names[class_id]
                confidence = confidences[i]
                logger.info(f"Found {class_name} with confidence {confidence:.2f}")
        
        visualize_results(rendered_image, save_path="advanced_result.jpg")
            
    except Exception as e:
        logger.error(f"An error occurred: {str(e)}")

2025-04-20 14:39:14,093 - INFO - Loading model from model.pt
2025-04-20 14:39:14,198 - INFO - Running inference with configuration: {'image_size': 640, 'conf_threshold': 0.25, 'iou_threshold': 0.45, 'verbose': False}
